In [1]:
import os

manager_name = api_key = os.environ.get("VINE_MANAGER_NAME")
print(f"Manager name: {manager_name}")

ports_str = os.environ.get("VINE_MANAGER_PORTS", "9123, 9150")
ports = [int(p.strip()) for p in ports_str.split(",")]
print(f"Manager Ports: {ports}")

Manager name: floability-4aec6052-7010-4b0c-85e5-eb7af34ba1df
Manager Ports: [502, 510]


In [2]:
import os
from coffea.nanoevents import NanoEventsFactory
from coffea import processor

import dask_awkward as dak
import hist.dask as hda

import warnings
warnings.filterwarnings("error", module="coffea.*")


use_taskvine = True

if use_taskvine:
    from ndcctools.taskvine import DaskVine

    vine_manager = DaskVine(ports, name=manager_name)

    vine_manager.disable_peer_transfers()

    executor_args = {
        "scheduler": vine_manager,
        "worker_transfers": True,
        # "task_mode": "function-calls",
    }
else:
    from distributed import Client
    client = Client()

    executor_args = {}

In [3]:
from coffea.nanoevents import NanoEventsFactory
from coffea.nanoevents.schemas import NanoAODSchema
import hist.dask as hda
import os

data_abs_path = os.path.abspath("data/small_data.root")

NanoAODSchema.warn_missing_crossrefs = False

events = (
    NanoEventsFactory.from_root(
        {data_abs_path: "Events"},        
        schemaclass=NanoAODSchema,
        metadata={"dataset": "SingleMu"},
        mode="dask"
    )
    .events()
)

q1_hist = (
    hda.Hist.new.Reg(100, 0, 200, name="met",
                     label=r"$E_{T}^{\text{miss}}\;[\mathrm{GeV}]$")
    .Double()
    .fill(events.MET.pt)
)

q1_hist.compute(**executor_args).plot1d()

Output()

task id 1 key ('hist-on-block-60fe7499400ff48d8771ad7fe3ab0697', 0) failed: output missing. 4 attempts left.
ERROR: ld.so: object '/usr/lib64/libauthbind.so.1' from LD_PRELOAD cannot be preloaded (cannot open shared object 
file): ignored.
ERROR: ld.so: object '/usr/lib64/libauthbind.so.1' from LD_PRELOAD cannot be preloaded (cannot open shared object 
file): ignored.
Traceback (most recent call last):
  File 
"/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/ndcctools/taskvine/dask_execut
or.py", line 763, in execute_graph_vertex
    return dask_task(task_args)
  File "/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/dask/_task_spec.py", 
line 755, in __call__
    return self.func(*new_argspec)
           ~~~~~~~~~^^^^^^^^^^^^^^
  File "/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/dask/_task_spec.py", 
line 203, in _execute_subgraph
    res = execute_graph(final, keys=)
  File "/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/dask/_task_spec.py", 
line 1027, in execute_graph
    cache = node(cache)
                 ~~~~^^^^^^^
  File "/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/dask/_task_spec.py", 
line 755, in __call__
    return self.func(*new_argspec)
           ~~~~~~~~~^^^^^^^^^^^^^^
  File "/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/uproot/_dask.py", line 
1379, in __call__
    result, _ = self._call_impl(
                ~~~~~~~~~~~~~~~^
        file_path, object_path, i_step_or_start, n_steps_or_stop, is_chunk
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    )
    ^
  File "/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/uproot/_dask.py", line 
1302, in _call_impl
    ttree = uproot._util.regularize_object_path(
        file_path,
    ...<3 lines>...
        self.real_options,
    )
  File "/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/uproot/_util.py", line 
970, in regularize_object_path
    file = ReadOnlyFile(
           ~~~~~~~~~~~~^
        file_path,
        ^^^^^^^^^^
    ...<3 lines>...
        **options,
        ^^^^^^^^^^
    ).root_directory
    ^
  File "/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/uproot/reading.py", 
line 573, in __init__
    self._begin_chunk = self._source.chunk(
                        ~~~~~~~~~~~~~~~~~~^
        0, self._options["begin_chunk_size"]
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ).detach_memmap()
    ^
  File 
"/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/uproot/source/fsspec.py", line
95, in chunk
    data = self._fs.cat_file(self._file_path, start=start, end=stop)
  File "/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/fsspec/spec.py", line 
797, in cat_file
    with self.open(path, "rb", **kwargs) as f:
         ~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^
  File "/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/fsspec/spec.py", line 
1338, in open
    f = self._open(
        path,
    ...<4 lines>...
        **kwargs,
    )
  File 
"/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/fsspec/implementations/local.p
y", line 210, in _open
    return LocalFileOpener(path, mode, fs=self, **kwargs)
  File 
"/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/fsspec/implementations/local.p
y", line 387, in __init__
    self._open()
    ~~~~~~~~~~^^
  File 
"/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/fsspec/implementations/local.p
y", line 392, in _open
    self.f = open(self.path, mode=self.mode)
             ~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: 
'/home/mdsaiful.islam/floability-examples/cms-physics-coffea/workflow/data/small
_data.root'

NoneType: None
Traceback (most recent call last):
  File "/srv/worker-67347-33/task.1/w_0179b49f-b244-4be0-b3fd-f0f931575758", line 27, in <module>
    raise error
  File "/srv/worker-67347-33/task.1/w_0179b49f-b244-4be0-b3fd-f0f931575758", line 13, in <module>
    exec_out = exec_function(*args, **kwargs)
  File "/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/ndcctools/taskvine/dask_executor.py", line 763, in 
execute_graph_vertex
   

2025/07/15 18:36:46.31 vine_manager[794197]notice: Cannot stat file file-rnd-jhkgjhrmiuqkgjp(temp-local-9F61CD8F-8CDA-4F17-8D7B-655E16C2B3F1): No such file or directory
2025/07/15 18:36:49.00 vine_manager[794197]notice: Cannot stat file file-rnd-wdzjjlhmanjwqlt(temp-local-D156E7A6-3F37-4C81-AF95-CC6F9FD21A45): No such file or directory


task id 3 key ('hist-on-block-60fe7499400ff48d8771ad7fe3ab0697', 0) failed: output missing. 2 attempts left.
ERROR: ld.so: object '/usr/lib64/libauthbind.so.1' from LD_PRELOAD cannot be preloaded (cannot open shared object 
file): ignored.
ERROR: ld.so: object '/usr/lib64/libauthbind.so.1' from LD_PRELOAD cannot be preloaded (cannot open shared object 
file): ignored.
Traceback (most recent call last):
  File 
"/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/ndcctools/taskvine/dask_execut
or.py", line 763, in execute_graph_vertex
    return dask_task(task_args)
  File "/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/dask/_task_spec.py", 
line 755, in __call__
    return self.func(*new_argspec)
           ~~~~~~~~~^^^^^^^^^^^^^^
  File "/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/dask/_task_spec.py", 
line 203, in _execute_subgraph
    res = execute_graph(final, keys=)
  File "/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/dask/_task_spec.py", 
line 1027, in execute_graph
    cache = node(cache)
                 ~~~~^^^^^^^
  File "/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/dask/_task_spec.py", 
line 755, in __call__
    return self.func(*new_argspec)
           ~~~~~~~~~^^^^^^^^^^^^^^
  File "/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/uproot/_dask.py", line 
1379, in __call__
    result, _ = self._call_impl(
                ~~~~~~~~~~~~~~~^
        file_path, object_path, i_step_or_start, n_steps_or_stop, is_chunk
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    )
    ^
  File "/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/uproot/_dask.py", line 
1302, in _call_impl
    ttree = uproot._util.regularize_object_path(
        file_path,
    ...<3 lines>...
        self.real_options,
    )
  File "/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/uproot/_util.py", line 
970, in regularize_object_path
    file = ReadOnlyFile(
           ~~~~~~~~~~~~^
        file_path,
        ^^^^^^^^^^
    ...<3 lines>...
        **options,
        ^^^^^^^^^^
    ).root_directory
    ^
  File "/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/uproot/reading.py", 
line 573, in __init__
    self._begin_chunk = self._source.chunk(
                        ~~~~~~~~~~~~~~~~~~^
        0, self._options["begin_chunk_size"]
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ).detach_memmap()
    ^
  File 
"/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/uproot/source/fsspec.py", line
95, in chunk
    data = self._fs.cat_file(self._file_path, start=start, end=stop)
  File "/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/fsspec/spec.py", line 
797, in cat_file
    with self.open(path, "rb", **kwargs) as f:
         ~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^
  File "/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/fsspec/spec.py", line 
1338, in open
    f = self._open(
        path,
    ...<4 lines>...
        **kwargs,
    )
  File 
"/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/fsspec/implementations/local.p
y", line 210, in _open
    return LocalFileOpener(path, mode, fs=self, **kwargs)
  File 
"/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/fsspec/implementations/local.p
y", line 387, in __init__
    self._open()
    ~~~~~~~~~~^^
  File 
"/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/fsspec/implementations/local.p
y", line 392, in _open
    self.f = open(self.path, mode=self.mode)
             ~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: 
'/home/mdsaiful.islam/floability-examples/cms-physics-coffea/workflow/data/small
_data.root'

NoneType: None
Traceback (most recent call last):
  File "/srv/worker-67347-33/task.3/w_de0393c4-0697-445f-9168-0faab01fa624", line 27, in <module>
    raise error
  File "/srv/worker-67347-33/task.3/w_de0393c4-0697-445f-9168-0faab01fa624", line 13, in <module>
    exec_out = exec_function(*args, **kwargs)
  File "/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/ndcctools/taskvine/dask_executor.py", line 763, in 
execute_graph_vertex
   

task id 4 key ('hist-on-block-60fe7499400ff48d8771ad7fe3ab0697', 0) failed: output missing. 1 attempts left.
ERROR: ld.so: object '/usr/lib64/libauthbind.so.1' from LD_PRELOAD cannot be preloaded (cannot open shared object 
file): ignored.
ERROR: ld.so: object '/usr/lib64/libauthbind.so.1' from LD_PRELOAD cannot be preloaded (cannot open shared object 
file): ignored.
Traceback (most recent call last):
  File 
"/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/ndcctools/taskvine/dask_execut
or.py", line 763, in execute_graph_vertex
    return dask_task(task_args)
  File "/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/dask/_task_spec.py", 
line 755, in __call__
    return self.func(*new_argspec)
           ~~~~~~~~~^^^^^^^^^^^^^^
  File "/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/dask/_task_spec.py", 
line 203, in _execute_subgraph
    res = execute_graph(final, keys=)
  File "/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/dask/_task_spec.py", 
line 1027, in execute_graph
    cache = node(cache)
                 ~~~~^^^^^^^
  File "/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/dask/_task_spec.py", 
line 755, in __call__
    return self.func(*new_argspec)
           ~~~~~~~~~^^^^^^^^^^^^^^
  File "/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/uproot/_dask.py", line 
1379, in __call__
    result, _ = self._call_impl(
                ~~~~~~~~~~~~~~~^
        file_path, object_path, i_step_or_start, n_steps_or_stop, is_chunk
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    )
    ^
  File "/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/uproot/_dask.py", line 
1302, in _call_impl
    ttree = uproot._util.regularize_object_path(
        file_path,
    ...<3 lines>...
        self.real_options,
    )
  File "/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/uproot/_util.py", line 
970, in regularize_object_path
    file = ReadOnlyFile(
           ~~~~~~~~~~~~^
        file_path,
        ^^^^^^^^^^
    ...<3 lines>...
        **options,
        ^^^^^^^^^^
    ).root_directory
    ^
  File "/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/uproot/reading.py", 
line 573, in __init__
    self._begin_chunk = self._source.chunk(
                        ~~~~~~~~~~~~~~~~~~^
        0, self._options["begin_chunk_size"]
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ).detach_memmap()
    ^
  File 
"/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/uproot/source/fsspec.py", line
95, in chunk
    data = self._fs.cat_file(self._file_path, start=start, end=stop)
  File "/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/fsspec/spec.py", line 
797, in cat_file
    with self.open(path, "rb", **kwargs) as f:
         ~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^
  File "/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/fsspec/spec.py", line 
1338, in open
    f = self._open(
        path,
    ...<4 lines>...
        **kwargs,
    )
  File 
"/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/fsspec/implementations/local.p
y", line 210, in _open
    return LocalFileOpener(path, mode, fs=self, **kwargs)
  File 
"/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/fsspec/implementations/local.p
y", line 387, in __init__
    self._open()
    ~~~~~~~~~~^^
  File 
"/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/fsspec/implementations/local.p
y", line 392, in _open
    self.f = open(self.path, mode=self.mode)
             ~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: 
'/home/mdsaiful.islam/floability-examples/cms-physics-coffea/workflow/data/small
_data.root'

NoneType: None
Traceback (most recent call last):
  File "/srv/worker-67347-33/task.4/w_4b4664e5-41d6-46a8-8bb2-1c0b8a5eaac0", line 27, in <module>
    raise error
  File "/srv/worker-67347-33/task.4/w_4b4664e5-41d6-46a8-8bb2-1c0b8a5eaac0", line 13, in <module>
    exec_out = exec_function(*args, **kwargs)
  File "/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/ndcctools/taskvine/dask_executor.py", line 763, in 
execute_graph_vertex
   

task id 5 key ('hist-on-block-60fe7499400ff48d8771ad7fe3ab0697', 0) failed: output missing. 0 attempts left.
ERROR: ld.so: object '/usr/lib64/libauthbind.so.1' from LD_PRELOAD cannot be preloaded (cannot open shared object 
file): ignored.
ERROR: ld.so: object '/usr/lib64/libauthbind.so.1' from LD_PRELOAD cannot be preloaded (cannot open shared object 
file): ignored.
Traceback (most recent call last):
  File 
"/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/ndcctools/taskvine/dask_execut
or.py", line 763, in execute_graph_vertex
    return dask_task(task_args)
  File "/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/dask/_task_spec.py", 
line 755, in __call__
    return self.func(*new_argspec)
           ~~~~~~~~~^^^^^^^^^^^^^^
  File "/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/dask/_task_spec.py", 
line 203, in _execute_subgraph
    res = execute_graph(final, keys=)
  File "/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/dask/_task_spec.py", 
line 1027, in execute_graph
    cache = node(cache)
                 ~~~~^^^^^^^
  File "/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/dask/_task_spec.py", 
line 755, in __call__
    return self.func(*new_argspec)
           ~~~~~~~~~^^^^^^^^^^^^^^
  File "/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/uproot/_dask.py", line 
1379, in __call__
    result, _ = self._call_impl(
                ~~~~~~~~~~~~~~~^
        file_path, object_path, i_step_or_start, n_steps_or_stop, is_chunk
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    )
    ^
  File "/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/uproot/_dask.py", line 
1302, in _call_impl
    ttree = uproot._util.regularize_object_path(
        file_path,
    ...<3 lines>...
        self.real_options,
    )
  File "/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/uproot/_util.py", line 
970, in regularize_object_path
    file = ReadOnlyFile(
           ~~~~~~~~~~~~^
        file_path,
        ^^^^^^^^^^
    ...<3 lines>...
        **options,
        ^^^^^^^^^^
    ).root_directory
    ^
  File "/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/uproot/reading.py", 
line 573, in __init__
    self._begin_chunk = self._source.chunk(
                        ~~~~~~~~~~~~~~~~~~^
        0, self._options["begin_chunk_size"]
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ).detach_memmap()
    ^
  File 
"/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/uproot/source/fsspec.py", line
95, in chunk
    data = self._fs.cat_file(self._file_path, start=start, end=stop)
  File "/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/fsspec/spec.py", line 
797, in cat_file
    with self.open(path, "rb", **kwargs) as f:
         ~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^
  File "/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/fsspec/spec.py", line 
1338, in open
    f = self._open(
        path,
    ...<4 lines>...
        **kwargs,
    )
  File 
"/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/fsspec/implementations/local.p
y", line 210, in _open
    return LocalFileOpener(path, mode, fs=self, **kwargs)
  File 
"/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/fsspec/implementations/local.p
y", line 387, in __init__
    self._open()
    ~~~~~~~~~~^^
  File 
"/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/fsspec/implementations/local.p
y", line 392, in _open
    self.f = open(self.path, mode=self.mode)
             ~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: 
'/home/mdsaiful.islam/floability-examples/cms-physics-coffea/workflow/data/small
_data.root'

NoneType: None
Traceback (most recent call last):
  File "/srv/worker-67347-33/task.5/w_bafc8412-8057-44dd-9112-5c9cd955c69a", line 27, in <module>
    raise error
  File "/srv/worker-67347-33/task.5/w_bafc8412-8057-44dd-9112-5c9cd955c69a", line 13, in <module>
    exec_out = exec_function(*args, **kwargs)
  File "/srv/tmp.lHg916P0mC/lib/python3.13/site-packages/ndcctools/taskvine/dask_executor.py", line 763, in 
execute_graph_vertex
   

2025/07/15 18:36:51.69 vine_manager[794197]notice: Cannot stat file file-rnd-ibytxtdaxgyhtgs(temp-local-6FCC42EE-22AB-4062-865C-9471D516971A): No such file or directory
2025/07/15 18:36:54.35 vine_manager[794197]notice: Cannot stat file file-rnd-ngixelxwmjweapu(temp-local-1CFFD4D9-82B9-4B09-8864-1A6AF89D7C97): No such file or directory
2025/07/15 18:36:57.07 vine_manager[794197]notice: Cannot stat file file-rnd-vbtbaybbiuvkmat(temp-local-61244314-0757-4366-B9B0-87B27C8943B5): No such file or directory


Exception: tasks for key ('hist-on-block-60fe7499400ff48d8771ad7fe3ab0697', 0) failed permanently